In [1404]:
#!unzip "/content/drive/MyDrive/train_dataset_Самолет2.zip" -d "/content/"

# Устанавливаем КБ и подгружаем библиотеки


In [1405]:
!pip install catboost

In [1406]:
import pandas as pd
import numpy as np
import random,os

# Проставляем сиды

In [1407]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(0)

# Подгружаем модели и обработанные тестовые данные

In [1408]:
import pickle

In [1409]:
best_models=[]

In [1410]:
for i in range(3):
  with open(f'model{i}.pkl','rb') as f:
    best_models.append(pickle.load(f))

test_data=np.loadtxt('test_data.txt')

In [1411]:
armatura=29
inventory=0
bal=0

In [1412]:
test_prices=pd.read_excel('test.xlsx')

In [1413]:
best_zakup_possible=1174600

In [1414]:
orig_signal_cnt = random.randint(0,6)

signal_cnt=orig_signal_cnt
signal=False

buy_orders=[]

enough=False

for i,row in enumerate(test_data):

    if i>28-inventory-1:
      buy_order=0
      enough=True # На складе достаточно арматуры, чтобы она дотянула до конца срока и не испортилась
    else:
      buy_order=1 # Стандартная закупка (если нет изменения тренда)

    queue=[row[0]] # Первая цена для расчета

    for model in best_models:
        queue.append(model.predict(row)) # Предсказание следующих 3 цен моделью

    down_m=np.mean([queue[1],queue[0]]) # Средний уровень первых 2х цен
    up_m=np.mean([queue[3],queue[2]]) # Средний уровень вторых 2х цен

    if up_m>down_m:
        signal=True # Если 2ой средний уровень больше первого, то ставим сигнал и ждем его подтверждение

    if signal:
        signal_cnt-=1 # Счетчик сигнала

    if signal_cnt==0:
        if np.mean([queue[1]-queue[0],queue[2]-queue[1],queue[3]-queue[2]])>0 and not enough: # По истечению счетчика если в среднем модель предсказывает везде рост и арматура нужна, то закупаем максимальное кол-во, которое не испортится
            buy_order=10-inventory
            print('trend change')

        signal=False
        signal_cnt=orig_signal_cnt # Ресет сигнала


    bal-=buy_order*test_prices['Цена на арматуру'].values[i]
    inventory+=buy_order # Добавляем арматуру на склад и вычитаем деньги

    buy_orders.append(buy_order)

    buy_order=1
    inventory-=1


print(bal,inventory) #Баланс и кол-во арматуры на складе
print(bal/best_zakup_possible) # Соотношение сколько потратили мы vs сколько потратил бы идеальный алгоритм

trend change
-1177300 3
-1.0022986548612294


In [1415]:
buy_orders=buy_orders+[0,0,0]

In [1416]:
sub=pd.read_excel('test.xlsx')
sub['Объем']=buy_orders

In [1417]:
sub.to_excel('submission.xlsx', index=False)

In [1418]:
sub

,dt,Цена на арматуру,Объем
0,2022-09-05,46000,1
1,2022-09-12,46000,1
2,2022-09-19,46000,1
3,2022-09-26,45500,1
4,2022-10-03,45500,1
5,2022-10-10,45000,1
6,2022-10-17,44000,1
7,2022-10-24,43000,1
8,2022-10-31,42000,1
9,2022-11-07,41500,1
